In [1]:
import pandas as pd
import datetime as dt

In [2]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
# df_bn5 = pd.read_csv("data/historical/GoldM-Oct-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 
df_bn5 = pd.read_csv("data/historical/GoldM-Oct-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [3]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

#### Apply rolling mean columns for data

In [4]:
df_bn5['SMA6'] = df_bn5['Close'].rolling(6).mean()
df_bn5['SMA21'] = df_bn5['Close'].rolling(21).mean()

## Input the evaluation date below

In [5]:
eval_date = "2020-09-08"

#### Extract a separate dataframe for the evaluation date

In [6]:
today_df_bn5 = df_bn5[eval_date]

In [7]:
today_df_bn5.head(5)

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-09-08 09:00:05,51000,50810,51000,50906,387,51103.000000,51008.857143
2020-09-08 09:05:00,50908,50898,50931,50925,235,51067.833333,51007.190476
2020-09-08 09:10:10,50921,50914,50930,50922,169,51031.500000,51006.380952
2020-09-08 09:15:02,50922,50916,50931,50920,101,50992.166667,51004.952381
2020-09-08 09:20:04,50923,50915,50925,50917,70,50953.333333,51003.380952


### Strategy function

In [12]:
def backtest_Intraday_5min_strategy(df):
    #Parameters
    STOPLOSS = 100
    SMA6_dead_band = 30
    
    #Other Variables
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("09:30", "%H:%M").time()
    close_time = dt.datetime.strptime("23:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("23:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE\t\t",  "Points\t",)
    
    for row in df.itertuples():
        SMA6_upper_band = row.SMA6 + SMA6_dead_band
        SMA6_lower_band = row.SMA6 - SMA6_dead_band
        
        # SELL STOPLOSS
        if row.Index.time() >= start_time and row.Index.time() <= close_time: 
            points = 0
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - row.Close
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL STOPLOSS\t", round(points,2))
            
            # BUY STOPLOSS
            points = 0
            if BUY and row.Close < BUY_STOPLOSS:
                points = row.Close - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY STOPLOSS\t", round(points,2))
            
            # SELL ENTRY
            points = 0
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY\t", round(points,2))
             
            # BUY ENTRY
            points = 0
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY\t", round(points,2))
            
            # DAY Close
            points = 0    
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY DAYCLOSE\t", round(points,2))

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL DAYCLOSE\t", round(points,2))
                    
                    
    
    print("\n\nTotal Signals:\t\t", Total_Signals)
    print("Total Sell Signals:\t", Total_Buy_Signal)
    print("Total Buy Signals:\t", Total_Sell_Signal)
    print("Total Points Gained:\t", round(GAIN_POINTS,2))

In [13]:
backtest_Intraday_5min_strategy(today_df_bn5)

Time			 Close		 SMA6		 SMA21		 SIGNAL	 TYPE		 Points	
2020-09-08 11:15:04 	 50999 	 50960.33 	 50940.48 	 BUY	 BUY ENTRY	 0
2020-09-08 14:20:10 	 51020 	 51051.5 	 51063.1 	 SELL	 SELL ENTRY	 21
2020-09-08 17:30:01 	 50935 	 50877.0 	 50906.43 	 BUY	 BUY ENTRY	 85
2020-09-08 18:05:18 	 50911 	 50941.17 	 50913.62 	 SELL	 SELL ENTRY	 -24
2020-09-08 19:35:04 	 50842 	 50807.67 	 50870.9 	 BUY	 BUY ENTRY	 69


Total Signals:		 5
Total Sell Signals:	 3
Total Buy Signals:	 2
Total Points Gained:	 151
